# Champions of sentiment discourse

AUTHOR: Michal Mochtak (michal.mochtak@ru.nl), Peter Rupnik (peter.rupnik@ijs.si), Nikola Ljubešić

DATE: 2024-06-24

---

In this notebook we look into specific countries and their sentiment scores on speaker- and party-level.

On the first run, the data will be downloaded from the internet. In the next cell a function was prepared to filter the dataset by specific conditions (e.g. taking only the MPs that have a specific number of speeches on the record). In the next cells we will inspect two countries in a comparable time frame, Croatia and France, and then the entire corpus across full time span.

In [18]:
import pandas as pd
pd.set_option('display.max_rows', None)
from pathlib import Path
import seaborn as sns
from IPython.display import display
if not Path("speeches.csv.zip").exists():
    from os import system
    system("wget https://huggingface.co/datasets/5roop/parlasent_data/resolve/main/speeches.csv.zip")
df = pd.read_csv("speeches.csv.zip")

/tmp/ipykernel_3583528/258120034.py:9: DtypeWarning: Columns (10,11,12,13,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("speeches.csv.zip")


In [19]:
def calculate_sentiment(target="Speaker_name", *, country=None, term=None, filter=None):
    global df
    all_countries = df.country.unique().tolist()
    if country == None:
        country = input(f"Choose country from {all_countries} \n(empty for all): ")

    # Filtering
    # Select speeches from a specific country:
    c0 = df.country == country
    if country in ["all", ""]:
        c0 = pd.Series([True for i in df.country])
    # Keep only MPs
    c1 = (df.Speaker_MP == "MP")
    # Limit searches to speeches longer than 100 characters:
    c2 = df.char_length >= 100
    # Include only speakers with at least 10 speeches:
    gb = df[c0&c1&c2].groupby("Speaker_name").logits_pondered.count().reset_index()
    speakers_to_keep = gb.Speaker_name[gb.logits_pondered >= 10]
    c3 = df.Speaker_name.isin(speakers_to_keep)
    ndf = df[c0&c1&c2&c3]
    if term == None:
        print(f"Available terms:")
        display(ndf.groupby("Term").agg({
            "Date": [min, max, "count"],
        }).sort_values(("Date", "min")), clear=True, )
        term = input(f"Choose term from {ndf.Term.unique().tolist()} (empty for all): ")
    if term:
        c0 = ndf.Term == term
        nndf = ndf[c0].reset_index(drop=True)
    else:
        nndf = ndf
    def percentage_of_negative(l: pd.Series) -> float:
        return (l < 2.0).sum()/l.shape[0]
    gb2 = nndf.groupby([f"{target}"]).agg({
        "logits_pondered": [percentage_of_negative, "count"],
    }).reset_index()
    gb2.columns = f"{target} percentage_of_negative count".split()

    # gb2 = gb2.merge(gb1, on=f"{target}", how="left")
    gb2 = gb2.sort_values(by="percentage_of_negative", ascending=False)

    if filter:
        sigma = gb2["count"].std()
        mu = gb2["count"].median()
        q1, q3 = gb2["count"].quantile([0.25, 0.75]).values.tolist()
        if filter == "1sd":
            c = (gb2["count"] >= mu - sigma) & (gb2["count"] <= mu+sigma)
        elif filter == "2sd":
            c = (gb2["count"] >= mu - 2*sigma) & (gb2["count"] <= mu + 2*sigma)
        elif filter == "interquartile":
            c = (gb2["count"] >= q1) & (gb2["count"] <= q3)
        elif filter == "Q1+":
            c = (gb2["count"] >= q1) & (gb2["count"] <= q3)
        else:
            raise NotImplementedError("Only accepts '1sd' or '2sd' for now")
        gb2 = gb2[c]
    return gb2.reset_index(drop=True)

Let's inspect the terms we have available, so that an approximately equal timeframe can be set:

In [20]:
df[df.country.isin(["HR", "FR"])].groupby("country Term Speaker_MP".split()).agg({
    "Date": [min, max, "count"]
}).sort_values(by=("Date", "min"))

/tmp/ipykernel_3583528/2170860219.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  df[df.country.isin(["HR", "FR"])].groupby("country Term Speaker_MP".split()).agg({
/tmp/ipykernel_3583528/2170860219.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df[df.country.isin(["HR", "FR"])].groupby("country Term Speaker_MP".split()).agg({


Date                    
                                           min         max   count
country Term            Speaker_MP                                
HR      5. mandat       MP          2003-12-22  2007-10-12   74856
                        notMP       2003-12-22  2007-10-12    4404
                        -           2004-04-01  2007-10-11    2632
        6. mandat       notMP       2008-01-11  2011-10-28    4200
                        MP          2008-01-11  2011-10-28   68561
                        -           2008-02-21  2011-10-27     455
        7. mandat       notMP       2011-12-22  2015-09-24    3824
                        MP          2011-12-22  2015-09-25   96544
                        -           2012-01-27  2015-07-03    1650
        8. mandat       MP          2015-12-03  2016-06-20   10559
                        notMP       2016-01-22  2016-06-20     477
                        -           2016-01-28  2016-03-02     110
        9. mandat       notMP       2016-10-14  2020-05-13    8002
                        MP          2016-10-14  2020-05-13  124817
                        -           2016-12-13  2020-02-05     791
FR      15e législature MP          2017-06-27  2022-03-23  413469
                        -           2017-06-27  2022-03-01   17765
                        notMP       2017-06-28  2022-03-23  283175
HR      10. mandat      MP          2020-07-22  2022-07-15   95051
                        notMP       2020-07-23  2022-07-15    5838
                        -           2020-12-17  2022-02-11    1563
FR      16e législature notMP       2022-06-28  2022-06-28      13
                        MP          2022-06-28  2022-06-29      17

In [21]:
calculate_sentiment("Speaker_name", country="HR", term="9. mandat").head()

,Speaker_name,percentage_of_negative,count
0,"Esih, Bruna",1.000000,13
1,"Glasnović, Željko",0.973545,567
2,"Klarin, Ivan",0.942857,35
3,"Pernar, Ivan",0.939297,1565
4,"Bunjac, Branimir",0.875089,1401


In [22]:
calculate_sentiment("Speaker_name", country="FR", term="15e législature").head()

,Speaker_name,percentage_of_negative,count
0,"Parigi, Jean-François",0.928571,14
1,"Dupont-Aignan, Nicolas",0.926335,543
2,"Houplain, Myriane",0.923077,13
3,"Cornut-Gentille, François",0.857143,84
4,"Evrard, José",0.846154,26


In [23]:
calculate_sentiment("Speaker_party", country="HR", term="9. mandat")


,Speaker_party,percentage_of_negative,count
0,Živi zid,0.894138,3429
1,HRAST,0.791436,724
2,PH,0.767142,1473
3,HSS,0.677419,1674
4,MOST,0.656041,10446
5,SDP,0.623890,14634
6,HSLS,0.615385,13
7,SNAGA,0.601911,628
8,HSU,0.588800,625
9,IDS,0.477612,469


In [24]:
calculate_sentiment("Speaker_party", country="FR", term="15e législature")

,Speaker_party,percentage_of_negative,count
0,FI,0.781236,12982
1,GDR,0.694278,6519
2,UDI_I,0.659779,2710
3,-,0.641280,6250
4,NG,0.630577,1838
5,LT,0.602067,4355
6,LR,0.521784,27360
7,LC,0.517361,576
8,EDS,0.513178,645
9,UDI-A-I,0.492188,896


# Overall most negative and most positive parties



Top 50 most negative speakers:

In [25]:
calculate_sentiment("Speaker_name", country="", term="",filter="Q1+").head(50)


,Speaker_name,percentage_of_negative,count
0,"Montero Soler, Alberto",1.000000,45
1,"Abascal Conde, Santiago",0.990741,108
2,"Karakōstas, Eyaggelos",0.983051,59
3,"Božić-Talijan, Elena",0.981481,54
4,"Mpoyrnoys, Iōannīs",0.980769,52
5,"Şık, Ahmet",0.980000,50
6,"Radić, Gojko",0.978723,47
7,"Santamaría Ruiz, Luis",0.977778,45
8,"Komnīnaka, Maria",0.976923,130
9,"Chatzīsavvas, Chrīstos",0.976744,86


50 least negative speakers:

In [26]:
calculate_sentiment("Speaker_name", country="", term="",filter="Q1+").tail(50)


,Speaker_name,percentage_of_negative,count
6713,"Héjj, Dávid Ádám",0.022727,132
6714,"Gültekin, Selim",0.022727,88
6715,"Kara, Ákos",0.021739,92
6716,"Majkić, Aleksandra",0.021277,47
6717,"Hofinger, Manfred",0.020000,100
6718,"Cielēns, Jānis",0.020000,200
6719,"Lohfeyer, Rosa",0.020000,100
6720,"Marinova, Cornelia Dobreva",0.019608,102
6721,"Dragaš, Mirjana",0.019417,103
6722,"Aleksić, Irena",0.019231,52


50 most negative parties:

In [27]:
calculate_sentiment("Speaker_party_name", country="", term="",filter="Q1+").head(50)


,Speaker_party_name,percentage_of_negative,count
0,PZDD,0.929078,141
1,KV,0.897059,136
2,Vox,0.895965,1586
3,Emek Partisi,0.895735,211
4,Živi zid,0.894138,3429
5,"Grandi Autonomie e Libertà (Grande Sud, Popola...",0.888889,243
6,A Párbeszéd Magyarországért parlamenti frakció...,0.884211,285
7,Λαϊκός Σύνδεσμος - Χρυσή Αυγή,0.876892,2510
8,Candidatura d'Unitat Popular,0.871186,295
9,A Momentum Mozgalom parlamenti frakciója a 9. ...,0.867384,279


50 least negative parties:

In [28]:
calculate_sentiment("Speaker_party_name", country="", term="",filter="Q1+").tail(50)


,Speaker_party_name,percentage_of_negative,count
267,MoVimento 5 Stelle,0.294790,4990
268,Büyük Birlik Partisi,0.293333,150
269,Klub Poselski Kukiz’15,0.291810,4640
270,Siumut,0.290909,385
271,Lipa,0.289767,987
272,Депутатська група «Відновлення України»,0.283019,212
273,"Per le Autonomie (SVP, UV, PATT, UPT) - PSI",0.280125,639
274,Italia Viva-P.S.I.,0.268640,912
275,Inuit Ataqatigiit,0.260274,1022
276,Sininen tulevaisuus,0.254871,1899
